In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os
import zipfile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input/covid19-ct-scans'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
print(os.listdir("../input"))
filenames = os.listdir("../input/covid19-ct-scans/ct_scans")
print(filenames)
print(len(filenames))


In [ ]:
import nibabel as nib

from scipy import ndimage
#function to read the nii file
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan
#function to normalize the image
def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume
#function to resize the nii file
def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img
def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume

In [ ]:
#read the paths of the CT scans from the class directories
#file named corona.nii are + and file named radiopaedia.nii are pneumonia
#normal_ct_scans_paths = [
    #os.path.join(os.getcwd(), "../input/covid19-ct-scans/ct-scans", file)
    #for folder in os.listdir("../input/covid19-ct-scans/ct-scans")
#]
#path=os.listdir("../input/covid19-ct-scans/ct-scans")
#for folder in path
#print(folder)
import os
import numpy as np
#fichiers = [f for f in os.listdir('../input/covid19-ct-scans/ct-scans') if isfile(join('../input/covid19-ct-scans/ct-scans', f)]
#print(fichiers)
#for f in os.listdir('../input/covid19-ct-scans/ct-scans'):
    #print(f)
#filenames_ct_scans = os.listdir("../input/covid19-ct-scans/ct_scans")
####
print(os.getcwd())
abnormal_ct_scan_paths = [
    #os.path.join(os.getcwd(), "covid19-ct-scans/ct_scans", x)
    #os.listdir()
    "../input/covid19-ct-scans/ct_scans"+"/"+x
    for x in os.listdir("../input/covid19-ct-scans/ct_scans") if x[0]=='c'
]
print("----corona ct scan path----")
print(abnormal_ct_scan_paths)
#print(len(normal_scan_paths))
normal_ct_scan_paths = [
    #os.path.join(os.getcwd(), "covid19-ct-scans/ct_scans", x)
    "../input/covid19-ct-scans/ct_scans"+"/"+x
    for x in os.listdir("../input/covid19-ct-scans/ct_scans") if x[0]=='r'
]
print("----radiopaedia ct scan path----")
print(normal_ct_scan_paths)


In [ ]:
import numpy as np
import nibabel as nib
import os
from scipy import ndimage
#function to read the nii file
def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan
#function to normalize the image
def normalize(volume):
    """Normalize the volume"""
    min = -1000
    max = 400
    volume[volume < min] = min
    volume[volume > max] = max
    volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    return volume
#function to resize the nii file
def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    desired_depth = 64
    desired_width = 128
    desired_height = 128
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img
def process_scan(path):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    # Normalize
    volume = normalize(volume)
    # Resize width, height and depth
    volume = resize_volume(volume)
    return volume
#abnormal_scans = np.array([process_scan(path) for path in abnormal_ct_scan_paths])
for path in abnormal_ct_scan_paths:
    print(path)
#abnormal_scans = np.array([process_scan(path) for path in abnormal_ct_scan_paths])  
#normal_scans = np.array([process_scan(path) for path in normal_ct_scan_paths])

In [ ]:
r=read_nifti_file(abnormal_ct_scan_paths[0])
print(type(r))

In [ ]:
abnormal_scans = np.array([process_scan(path) for path in abnormal_ct_scan_paths])
normal_scans = np.array([process_scan(path) for path in normal_ct_scan_paths])

In [ ]:
print(type(abnormal_scans)) #numpy.ndarray

In [ ]:
abnormal_labels = np.array([1 for i in range(len(abnormal_scans))])
normal_labels = np.array([0 for i in range(len(normal_scans))])

In [ ]:
# Split data in the ratio 70-30 for training and validation.
x_train = np.concatenate((abnormal_scans[:7], normal_scans[:7]), axis=0)
y_train = np.concatenate((abnormal_labels[:7], normal_labels[:7]), axis=0)
x_val = np.concatenate((abnormal_scans[7:], normal_scans[7:]), axis=0)
y_val = np.concatenate((abnormal_labels[7:], normal_labels[7:]), axis=0)
print(
    "Number of samples in train and validation are %d and %d."
    % (x_train.shape[0], x_val.shape[0])
)

In [ ]:
import random

from scipy import ndimage


@tf.function
def rotate(volume):
    """Rotate the volume by a few degrees"""

    def scipy_rotate(volume):
        # define some rotation angles
        angles = [-20, -10, -5, 5, 10, 20]
        # pick angles at random
        angle = random.choice(angles)
        # rotate volume
        volume = ndimage.rotate(volume, angle, reshape=False)
        volume[volume < 0] = 0
        volume[volume > 1] = 1
        return volume

    augmented_volume = tf.numpy_function(scipy_rotate, [volume], tf.float32)
    return augmented_volume


def train_preprocessing(volume, label):
    """Process training data by rotating and adding a channel."""
    # Rotate volume
    volume = rotate(volume)
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


def validation_preprocessing(volume, label):
    """Process validation data by only adding a channel."""
    volume = tf.expand_dims(volume, axis=3)
    return volume, label


In [ ]:
# Define data loaders.
train_loader = tf.data.Dataset.from_tensor_slices((x_train, y_train))
validation_loader = tf.data.Dataset.from_tensor_slices((x_val, y_val))

batch_size = 2
# Augment the on the fly during training.
train_dataset = (
    train_loader.shuffle(len(x_train))
    .map(train_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)
# Only rescale.
validation_dataset = (
    validation_loader.shuffle(len(x_val))
    .map(validation_preprocessing)
    .batch(batch_size)
    .prefetch(2)
)


In [ ]:
import matplotlib.pyplot as plt

data = train_dataset.take(1)#returns nd array
images, labels = list(data)[0]
images = images.numpy()
image = images[0]
print("Dimension of the CT scan is:", image.shape)
plt.imshow(np.squeeze(image[:, :, 30]), cmap="gray")


In [ ]:
def plot_slices(num_rows, num_columns, width, height, data):
    """Plot a montage of 20 CT slices"""
    data = np.rot90(np.array(data))
    data = np.transpose(data)
    data = np.reshape(data, (num_rows, num_columns, width, height))
    rows_data, columns_data = data.shape[0], data.shape[1]
    heights = [slc[0].shape[0] for slc in data]
    widths = [slc.shape[1] for slc in data[0]]
    fig_width = 12.0
    fig_height = fig_width * sum(heights) / sum(widths)
    f, axarr = plt.subplots(
        rows_data,
        columns_data,
        figsize=(fig_width, fig_height),
        gridspec_kw={"height_ratios": heights},
    )
    for i in range(rows_data):
        for j in range(columns_data):
            axarr[i, j].imshow(data[i][j], cmap="gray")
            axarr[i, j].axis("off")
    plt.subplots_adjust(wspace=0, hspace=0, left=0, right=1, bottom=0, top=1)
    plt.show()


# Visualize montage of slices.
# 4 rows and 10 columns for 100 slices of the CT scan.
plot_slices(4, 10, 128, 128, image[:, :, :40])


In [ ]:
def get_model(width=128, height=128, depth=64):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = layers.MaxPool3D(pool_size=2)(x)
    x = layers.BatchNormalization()(x)

    x = layers.GlobalAveragePooling3D()(x)
    x = layers.Dense(units=512, activation="relu")(x)
    x = layers.Dropout(0.3)(x)

    outputs = layers.Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model

# Build model.
model = get_model(width=128, height=128, depth=64)
model.summary()


In [ ]:
# Compile model.
initial_learning_rate = 0.0001
lr_schedule = keras.optimizers.schedules.ExponentialDecay(
    initial_learning_rate, decay_steps=100000, decay_rate=0.96, staircase=True
)
model.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.Adam(learning_rate=lr_schedule),
    metrics=["acc"],
)

# Define callbacks.
checkpoint_cb = keras.callbacks.ModelCheckpoint(
    "3d_image_classification.h5", save_best_only=True
)
early_stopping_cb = keras.callbacks.EarlyStopping(monitor="val_acc", patience=15)

# Train the model, doing validation at the end of each epoch
epochs = 100
model.fit(
    train_dataset,
    validation_data=validation_dataset,
    epochs=epochs,
    shuffle=True,
    verbose=2,
    callbacks=[checkpoint_cb, early_stopping_cb],
)


In [ ]:
fig, ax = plt.subplots(1, 2, figsize=(20, 3))
ax = ax.ravel()

for i, metric in enumerate(["acc", "loss"]):
    ax[i].plot(model.history.history[metric])
    ax[i].plot(model.history.history["val_" + metric])
    ax[i].set_title("Model {}".format(metric))
    ax[i].set_xlabel("epochs")
    ax[i].set_ylabel(metric)
    ax[i].legend(["train", "val"])


In [ ]:
# Load best weights.
model.load_weights("3d_image_classification.h5")
prediction = model.predict(np.expand_dims(x_val[0], axis=0))[0]
scores = [1 - prediction[0], prediction[0]]

class_names = ["normal", "abnormal"]
for score, name in zip(scores, class_names):
    print(
        "This model is %.2f percent confident that CT scan is %s"
        % ((100 * score), name)
    )


In [ ]:
#ok
#######visual explanations#######
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import datetime
import numpy as np
#from deploy_config import*  #Import Paths and Model Config file
#from loss_funnction_And_matrics import*  #Import Loss functions
#from Resnet_3D import Resnet3D #import the model
import pandas as pd
from tensorflow.keras.optimizers import Adam
import cv2

In [ ]:
#ok
INPUT_PATCH_SIZE=(128,128,64,1)
inputs = tf.keras.Input(shape=INPUT_PATCH_SIZE, name='CT')
#Model_3D=3dcnn(inputs,num_classes=2) # Load Model
mod=get_model(128,128,64)
Model_3D=mod(inputs,num_classes=2)
Model_3D.load_weights('3d_image_classification.h5') #Load Weights
#print('Loading The Model from this path--{}'.format(MODEL_PATH))
Model_3D.summary()
###---lAYER-Name--to-visualize--###
LAYER_NAME='conv3d_3'
# Create a graph that outputs target convolution and output
grad_model = tf.keras.models.Model([Model_3D.inputs], [Model_3D.get_layer(LAYER_NAME).output, Model_3D.output])
grad_model.summary()

In [ ]:
NUM_OF_GPU=1
DISTRIIBUTED_STRATEGY_GPUS=["gpu:0","gpu:1","gpu:2"]
##Network Configuration
NUMBER_OF_CLASSES=2
INPUT_PATCH_SIZE=(128,128,62, 2)
TRAIN_NUM_RES_UNIT=3
TRAIN_NUM_FILTERS=(64 ,64, 128, 256)
TRAIN_STRIDES=( (2, 2, 2), (2, 2, 2), (2, 2, 2))
TRAIN_CLASSIFY_ACTICATION=tf.nn.relu6
TRAIN_KERNAL_INITIALIZER=tf.keras.initializers.VarianceScaling(distribution='uniform')
TRAIN_CLASSIFY_LEARNING_RATE =0.0001
#TRAIN_CLASSIFY_LOSS=Weighted_BCTL
#3d_image_classification.h5
OPTIMIZER=tf.keras.optimizers.Adam(lr=TRAIN_CLASSIFY_LEARNING_RATE,epsilon=1e-5)
TRAIN_CLASSIFY_METRICS=tf.keras.metrics.acc()


In [ ]:
#ok
def Get_Build_model(Input_patch_size,Model_weight,Layer_name):
    inputs = tf.keras.Input(shape=Input_patch_size, name='CT')
    #Model_3D=Resnet3D(inputs,num_classes=NUMBER_OF_CLASSES)
    Model_3D=get_model(128,128,64)
    Model_3D.load_weights("3d_image_classification.h5")
    #Model_3D.load_weights(Model_weight)
    #print('Loading The Model from this path--{}'.format(MODEL_PATH))
    Model_3D.summary()
    Build_model=tf.keras.models.Model([Model_3D.inputs], [Model_3D.get_layer(Layer_name).output, Model_3D.output])
    return Build_model

In [ ]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
import os
import datetime
import numpy as np
import pandas as pd
import SimpleITK as sitk
import math
import cv2
#from deploy_config import*
#from loss_funnction_And_matrics import*
#from Resnet_3D import Resnet3D
from tensorflow.keras.optimizers import Adam
from matplotlib import pyplot as plt
from skimage.transform import resize
#from Guided_GradCAM_3D_config import*

In [ ]:
#ok
# Function to get the image chunk fot guided GradCAM
def Get_image_array_Array_and_give_chunk(image_array,patch_slice_slice):

    Devide_integer=image_array.shape[0] // patch_slice_slice
    Reminder= image_array.shape[0] % patch_slice_slice
    print('CT Volume_Shape={}'.format(image_array.shape))
    print('Devide_integer={}'.format(Devide_integer))
    print('Reminder={}'.format(Reminder))
    print('Total of {} + {} ={} Should ={}'.format(patch_slice_slice*Devide_integer,Reminder,patch_slice_slice*Devide_integer+Reminder,image_array.shape[0]))

    lastpatch_starts_from= (image_array.shape[0])-patch_slice_slice
    print(lastpatch_starts_from)

    patch_list=[]
    patch_start=0
    patch_end=patch_slice_slice
    for i in range(Devide_integer):
        #print(patch_start)
        #print(patch_end)
        ct_volume=image_array[patch_start:patch_end,:,:]
        #print(ct_volume.shape)
        patch_list.append(ct_volume)
        patch_start+=patch_slice_slice
        patch_end+=patch_slice_slice

    last_slice_number_would_be=image_array.shape[0]
    print(last_slice_number_would_be)
    last_patch_When_making_nifty=(patch_slice_slice)-Reminder
    print(last_patch_When_making_nifty)
    Slice_will_start_from_here=last_slice_number_would_be-patch_slice_slice
    print(Slice_will_start_from_here)
    last_patch=image_array[Slice_will_start_from_here:,:,:]
    last_patch.shape
    patch_list.append(last_patch)

    return patch_list,last_patch_When_making_nifty

In [ ]:
#ok
def Guided_GradCAM_3D(Grad_model,ct_io,Class_index):
    # Create a graph that outputs target convolution and output
    grad_model = Grad_model
    input_ct_io=tf.expand_dims(ct_io, axis=-1)
    input_ct_io=tf.expand_dims(input_ct_io, axis=0)
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(input_ct_io)
        loss = predictions[:, Class_index]
    # Extract filters and gradients
    output = conv_outputs[0]
    grads = tape.gradient(loss, conv_outputs)[0]

    ##--Guided Gradient Part
    gate_f = tf.cast(output > 0, 'float32')
    gate_r = tf.cast(grads > 0, 'float32')
    guided_grads = tf.cast(output > 0, 'float32') * tf.cast(grads > 0, 'float32') * grads

    # Average gradients spatially
    weights = tf.reduce_mean(guided_grads, axis=(0, 1,2))
    # Build a ponderated map of filters according to gradients importance
    cam = np.ones(output.shape[0:3], dtype=np.float32)
    for index, w in enumerate(weights):
        cam += w * output[:, :, :, index]

    capi=resize(cam,(ct_io.shape))
    print(capi.shape)
    capi = np.maximum(capi,0)
    heatmap = (capi - capi.min()) / (capi.max() - capi.min())
    return heatmap

In [ ]:
#ok
def generate_guided_grad_cam(Nifti_path,Model_weight,Class_index,Input_patch_size_slice_number,Layer_name,Save_path):
    # Reading the CT
    img_path=Nifti_path
    Class_index=Class_index
    Model_weight=Model_weight
    Layer_name=Layer_name
    img_sitk = sitk.ReadImage(img_path, sitk.sitkFloat32)
    image= sitk.GetArrayFromImage(img_sitk)
    Input_patch_size=[Input_patch_size_slice_number,image.shape[1],image.shape[2],1]

    get_grad_model=Get_Build_model(Input_patch_size,Model_weight,Layer_name)
    ct_patch_chunk_List,last_patch_number=Get_image_array_Array_and_give_chunk(image_array=image,patch_slice_slice=Input_patch_size_slice_number)
    first_heatmap=Guided_GradCAM_3D(get_grad_model,ct_patch_chunk_List[0],Class_index=Class_index)
    heatmap_concat=first_heatmap
    for i in range(1,(len(ct_patch_chunk_List)-1)):
        from Resnet_3D import Resnet3D
        get_heatmap=Guided_GradCAM_3D(get_grad_model,ct_patch_chunk_List[i],Class_index=Class_index)
        heatmap_concat=np.concatenate((heatmap_concat, get_heatmap), axis=0)
    last_heatmap=Guided_GradCAM_3D(get_grad_model,ct_patch_chunk_List[-1],Class_index=Class_index)
    heatmap_concat=np.concatenate((heatmap_concat, last_heatmap[last_patch_number:,:,:]), axis=0)
    s_itk_image = sitk.GetImageFromArray(heatmap_concat)
    s_itk_image.CopyInformation(img_sitk)
    sitk.WriteImage(s_itk_image, Save_path)
    return 

In [ ]:
#ok
NIFTI_PATH='../input/covid19-ct-scans/ct_scans/coronacases_org_001.nii'
MODEL_WEIGHT="3d_image_classification.h5"
CLASS_INDEX=1
INPUT_PATCH_SIZE_SLICE_NUMBER=301
LAYER_NAME='conv3d_3'
SAVE_PATH='./'

In [ ]:
#ok last one
if __name__ == '__main__':
    img_path=NIFTI_PATH
    Model_weight=MODEL_WEIGHT
    Class_index=CLASS_INDEX
    Input_patch_size_slice_number=INPUT_PATCH_SIZE_SLICE_NUMBER
    Layer_name=LAYER_NAME
    Save_path=SAVE_PATH
    generate_guided_grad_cam(img_path,Model_weight,Class_index,Input_patch_size_slice_number,Layer_name,Save_path)

In [ ]:
MODEL_WEIGHT="Path/of/Model/Weight/XXX.h5"
CLASS_INDEX=2 # Index of the class for which you want to see the Guided-gradcam
INPUT_PATCH_SIZE_SLICE_NUMBER= ???# Input patch slice you want to feed at a time
LAYER_NAME='conv3d_18' # Name of the layer from where you want to get the Guided-GradCAM
NIFTI_PATH="imput/niftidata/path/XXX.nii.gz"
SAVE_PATH="/Output/niftydata/path/ML_Guided_GradCaN_XXXX.nii.gz"

In [ ]:
def resize_image_with_crop_or_pad(image, img_size=(64, 64, 64), **kwargs):
    """Image resizing. Resizes image by cropping or padding dimension
     to fit specified size.
    Args:
        image (np.ndarray): image to be resized
        img_size (list or tuple): new image size
        kwargs (): additional arguments to be passed to np.pad
    Returns:
        np.ndarray: resized image
    """

    assert isinstance(image, (np.ndarray, np.generic))
    assert (image.ndim - 1 == len(img_size) or image.ndim == len(img_size)), \
        'Example size doesnt fit image size'

    # Get the image dimensionality
    rank = len(img_size)

    # Create placeholders for the new shape
    from_indices = [[0, image.shape[dim]] for dim in range(rank)]
    to_padding = [[0, 0] for dim in range(rank)]

    slicer = [slice(None)] * rank

    # For each dimensions find whether it is supposed to be cropped or padded
    for i in range(rank):
        if image.shape[i] < img_size[i]:
            to_padding[i][0] = (img_size[i] - image.shape[i]) // 2
            to_padding[i][1] = img_size[i] - image.shape[i] - to_padding[i][0]
        else:
            from_indices[i][0] = int(np.floor((image.shape[i] - img_size[i]) / 2.))
            from_indices[i][1] = from_indices[i][0] + img_size[i]

        # Create slicer object to crop or leave each dimension
        slicer[i] = slice(from_indices[i][0], from_indices[i][1])

    # Pad the cropped image to extend the missing dimension
    return np.pad(image[slicer], to_padding, **kwargs)

In [ ]:
##----Load---Image
import SimpleITK as sitk
img_path='../input/covid19-ct-scans/ct_scans/coronacases_org_001.nii'
img_sitk = sitk.ReadImage(img_path, sitk.sitkFloat32)
image= sitk.GetArrayFromImage(img_sitk)
resized_img=resize_image_with_crop_or_pad(image=image,img_size=(128,128,64),mode='symmetric')
#resized_img=image
resized_img.shape
io_img=tf.expand_dims(resized_img, axis=-1)
print(io_img.shape)
io_img=tf.expand_dims(io_img, axis=0)
print(io_img.shape)
###----index of the class
CLASS_INDEX=2

###--Compute GRADIENT
with tf.GradientTape() as tape:
    conv_outputs, predictions = grad_model(io_img)
    loss = predictions[:, CLASS_INDEX]

# Extract filters and gradients
output = conv_outputs[0]
grads = tape.gradient(loss, conv_outputs)[0]

In [ ]:
# Average gradients spatially
weights = tf.reduce_mean(grads, axis=(0, 1,2))
# Build a ponderated map of filters according to gradients importance
cam = np.zeros(output.shape[0:3], dtype=np.float32)

for index, w in enumerate(weights):
    cam += w * output[:, :, :, index]

In [ ]:


from skimage.transform import resize
from matplotlib import pyplot as plt
capi=resize(cam,(128,128,64))
#print(capi.shape)
capi = np.maximum(capi,0)
heatmap = (capi - capi.min()) / (capi.max() - capi.min())
f, axarr = plt.subplots(2,3,figsize=(15,10));
f.suptitle('Grad-CAM')
slice_count=127
slice_count2=30
    
axial_ct_img=np.squeeze(resized_img[slice_count, :,:])
axial_grad_cmap_img=np.squeeze(heatmap[slice_count,:, :])

coronal_ct_img=np.squeeze(resized_img[:,slice_count2,:])
coronal_grad_cmap_img=np.squeeze(heatmap[:,slice_count2,:]) 

img_plot = axarr[0,0].imshow(axial_ct_img, cmap='gray');
axarr[0,0].axis('off')
axarr[0,0].set_title('CT')
    
img_plot = axarr[0,1].imshow(axial_grad_cmap_img, cmap='jet');
axarr[0,1].axis('off')
axarr[0,1].set_title('Grad-CAM')
    
axial_overlay=cv2.addWeighted(axial_ct_img,0.3,axial_grad_cmap_img, 0.6, 0)
    
img_plot = axarr[0,2].imshow(axial_overlay,cmap='jet');
axarr[0,2].axis('off')
axarr[0,2].set_title('Overlay')


img_plot = axarr[1,0].imshow(coronal_ct_img, cmap='gray');
axarr[1,0].axis('off')
axarr[1,0].set_title('CT')
    
img_plot = axarr[1,1].imshow(coronal_grad_cmap_img, cmap='jet');
axarr[1,1].axis('off')
axarr[1,1].set_title('Grad-CAM')
    
Coronal_overlay=cv2.addWeighted(coronal_ct_img,0.3,coronal_grad_cmap_img, 0.6, 0)
    
img_plot = axarr[1,2].imshow(Coronal_overlay,cmap='jet');
axarr[1,2].axis('off')
axarr[1,2].set_title('Overlay')